# Modelos 4: Tentando algo diferente
**Nomes**: Bruna Guedes Pereira, Laura Medeiros Dal Ponte, Mariana Melo Pereira

## Introdução

O aprendizado de máquina consolidou-se como uma das ferramentas mais influentes da ciência contemporânea, permitindo que dados complexos promovam decisões automatizadas. Dentro desse campo, os algoritmos de classificação ocupam papel central, pois possibilitam identificar padrões e organizar informações em categorias de forma precisa e eficiente.

Métodos clássicos como Support Vector Machine (SVM), Naive Bayes e Discriminant Analysis possuem relevância histórica na resolução de problemas de classificação, sendo ainda muito utilizados em diversos experimentos científicos. Explorar o funcionamento interno desses algoritmos, compreendendo suas bases matemáticas e avaliando seu desempenho em conjuntos de dados reais, é um exercício essencial de formação acadêmica.

Dessa forma, o presente trabalho dedica-se a investigar o algoritmo Naive Bayes, implementá-lo de maneira didática e comparar seus resultados a um modelo de referência. Trata-se de uma família diversa de algoritmos, sendo dividida em três tipos: Gaussian Naive Bayes; Multinomial Naive Bayes e Bernoulli Naive Bayes. Aqui, focaremos apenas no Gaussian Naive Bayes, discutindo sua lógica e seu desempenho.


A seguir, uma breve contextualização "divertida" para ambientar e motivar os leitores a entender os fundamentos da análise realizada.

### 🎼 **Ricardo e a Harmonia das Fronteiras: A Balada de Bayes**

Ricardo nasceu em uma linhagem de cavaleiros honrados, treinado desde cedo para empunhar espadas e proteger reinos. Mas seu coração sempre bateu em outro ritmo — o das melodias. Inspirado pelas apresentações da Senhorita Creizuda e da Alfaiate Ligeirinha, ele sonhava em ser um **bardo**, em transformar sentimentos em canções e tocar almas com acordes sinceros.

Mesmo preso às expectativas de sua família, Ricardo nunca abandonou sua paixão. Nas noites calmas, ele se refugia na **Floresta Solitária**, onde canta sobre um amor que nunca desiste, nunca desilude, nunca fere. Suas canções ecoam como promessas à vida que deseja, mas que não pode viver por completo.

Foi em uma dessas noites que Ricardo encontrou um antigo manuscrito escondido sob uma raiz de árvore. Nele, estava descrito um algoritmo ancestral, usado por magos analíticos para **decifrar padrões ocultos com surpreendente simplicidade**: o **Naive Bayes**.

Ricardo aprendeu que o Naive Bayes é como um contador de histórias que confia em probabilidades para prever o rumo dos acontecimentos. Ele parte de uma suposição ousada — que cada característica de um dado é **independente das demais**, como notas que soam separadas, mas que juntas formam uma melodia coerente.

A cada nova observação, o algoritmo calcula a **probabilidade de pertencimento a cada classe**, como se perguntasse: “Qual é a chance de esta canção pertencer ao repertório da esperança, ou ao da saudade?” E mesmo com sua simplicidade, o Naive Bayes surpreende pela eficiência, especialmente quando os dados são vastos e as palavras são muitas — como em cartas, poemas ou canções.

Ricardo treinou um modelo Naive Bayes com dados reais, comparando seu desempenho com o de um classificador mais ingênuo. E descobriu que, como em suas baladas, **a beleza pode estar na simplicidade**, e que mesmo suposições aparentemente ingênuas podem revelar verdades profundas.

Mesmo que nunca possa ser um bardo aos olhos do mundo, Ricardo sabe que, com cada linha de código e cada nota tocada ao luar, ele está escrevendo sua própria balada — uma que fala de coragem, de aprendizado, e de nunca desistir daquilo que faz o coração vibrar.

### O algoritmo Gaussian Naive Bayes

O classificador Naive Bayes (NB) prevê dados baseando-se no Teorema de Bayes. Ele calcula a *probabilidade a posteriori* de cada uma das classes dado os features escolhidos, atribuindo os dados a classe (rótulo) com maior *probabilidade a posteriori*. O Teorema de Bayes é uma fórmula da probabilidade condicional que considere a probabilidade de um evento $A$ ocorrer dado que um evento $B$ ocorreu. Ela é escrita da seguinte forma: 
$$
P(A | B) = \frac{P(A) P(B | A).}{P(B)}.
$$
Onde $P(A)$ é a probabilidade inicial; $P(B|A)$ é a verossimilhança (probabilidade de $A$ dado que $B$ ocorreu); $P(B)$ é a probabilidade individual (também chamada de evidência); e $P(A|B)$ é a probabilidade a posteriori.

"Naive" quer dizer literalmente "Ingênuo", devido a lógica simplista do algoritmo ao assumir que todas as features são condicionalmente independentes umas das outras. Desse modo, cada observação serve como uma evidência independente no cálculo da probabilidade final de uma classe. Por exemplo: ele pode considerar que duas probabilidades são independentes uma da outra, quando na verdade elas podem estar correlacionadas. Apesar disso, essa simplificação resulta em bons resultados. Ele também atribui ás features numéricas uma Distribuição Gaussiana (ou Distribuição Normal), que sera melhor evidenciada mais adiante. 

## 🧑‍💻 Vamos, então, para o Código Mágico?

O Código Mágico, capaz de resolver o desafio proposto, será construído a partir da linguagem Python, em conjunto com as suas bibliotecas `scikit-learn`, `pandas`, `scipy` e `statistics`. O primeiro passo será, então, importar essas bibliotecas para o nosso ambiente de programação.

### Importação das bibliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import norm
import statistics as st

Como pré-requisito, o Naive Bayes é projetado para trabalhar com valores contínuos e distribuídos normalmente. Também devemos checar se as features de nosso DataSet são mesmo independentes umas das outras. Para essa demonstração, vamos utilizar o DataSet didático `IRIS`. Ele é um conjunto de dados simples, com quatro dados contínuos (largura e comprimento da sépala e largura e comprimento da pétala) e um dado categórico (nome da espécie). 

In [ ]:
df = pd.read_csv("IRIS.csv")
df

A primeira coisa que o algoritmo faz é calcular as médias ($μ$) e os desvios padrões ($σ$) de cada feature dentro das três classes. Abaixo, temos uma função que demonstra de forma mais visual como isso funciona:

In [ ]:
def dados(df, coluna_classe):

    groupby = df.groupby(coluna_classe)
    media = groupby.mean()
    dsvp = groupby.std()

    return media, dsvp

dados(df, "species")

Como consta no nome do algoritmo, também podemos atribuir uma **curva gaussiana** que representa a distribuição gaussiana de cada um dos features. O algoritmo usa uma função de densidade de probabilidade (PDF), que é dada por: 

$$
f(x) = \frac{1}{\sqrt{2 \pi \sigma}} \; e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

Tendo isso em mente, podemos comparar as curvas gaussianas que obtemos de cada média e desvio padrão resultante dos tributos. Abaixo, temos uma visualização do gráfico com as distribuições gaussianas referentes ao comprimento da pétala de cada uma das três espécies:

In [ ]:
media1 = 1.464 
media2 = 5.552
media3 = 4.260
desvio1 = 0.173511
desvio2 = 0.551895 
desvio3 = 0.469911


x =  np.linspace(media1 - 4*desvio1, media2 + 4*desvio2)

y1 = norm.pdf(x, media1, desvio1)
y2 = norm.pdf(x, media2, desvio2)
y3 = norm.pdf(x, media3, desvio3)

plt.plot(x, y1)
plt.plot(x, y2)
plt.plot(x, y3)

De forma simples, o algoritmo armazena esses dados para obter as curvas gaussianas. Ao deparar-se com dados novos, o mesmo aplica o Teorema de Bayes para atribuir rótulos. A partir do nosso dataset, podemos atribuir a seguinte lógica:

- P(A|B) é a probabilidade de que uma flor seja de uma espécie A, dado que as medidas de sépala e pétala coincidem com uma outra espécie B.
- P(A) é a probabilidade inicial de que essa flor seja da espécie A (sem nenhuma medida)
- P(B|A) é a verossimilhança; a probabilidade de que a flor tenha medidas de outra espécie B dado que ela já é uma flor da espécie A.
- P(B) é a probabilidade de que essas medidas estejam presentes em outras espécies de flor 

A probabilidade inicial parte de dados do próprio dataset; por exemplo, considerando que temos, ao todo, 150 flores para 3 espécies, e que cada espécie possui uma distribuição igual, cada uma possuí probabilidade inicial correspondente a ~33%. A partir disso, o algoritmo calcula a verossimilhança (P(A|B)). Devido a suposição *Naive*, ele a calcula multiplicando as quatro probabilidades iniciais. Por fim, ele utiliza o Teorema de Bayes para juntar a verossimilhança com a probabilidade inicial (que contêm nossa suposição da espécie). A maior probabilidade a posteriori é, portanto, considerada.

### Exemplo de classificação

A partir dos dados em nosso DataFrame, vamos prever uma certa amostra de dados, considerando a lógica mencionada acima:

In [ ]:
X = df.drop(["species"], axis=1)
y = df["species"]

gnb = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

gnb.fit(X_train, y_train)

y_previsto = gnb.predict(X_test)

y_previsto

### Exemplo com um dado novo

Supondo que obtemos informações sobre uma nova flor misteriosa, como será que o nosso algoritmo se comporta?

In [ ]:
X = df.drop(["species"], axis=1)
y = df["species"]

gnb = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

gnb.fit(X_train, y_train)

novos_valores = [6.1, 3.0, 4.9, 1.8]

amostra_nova = pd.DataFrame([novos_valores], columns=X.columns)

y_previsto = gnb.predict(X_test)

y_previsto[0] ## Como ele retorna um array

### A wild baseline appears!

Agora que conseguimos prever nossos dados com o classificador Gaussian NB, vamos realizar uma previsão com um algoritmo baseline e comparar seu desempenho com o nosso algoritmo. O algoritmo K-NN é um classificador baseline que prevê dados baseados no princípio de que semelhantes encontram-se próximos, tomando como parâmetros um número K de vizinhos (a quantidade de dados próximos que consideramos). 

In [ ]:
X = df[["sepal_length","sepal_width","petal_length"]] #Features
y = df[["species"]] #Target

## 1) Instanciar o modelo usando o algoritmo K-NN com o número de vizinhos desejados
modelo_knn = KNeighborsClassifier(n_neighbors=20)

## 2) Separar os dados em teste e treino
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3)

## 3) Treinar os dados
modelo_knn.fit(X_treino, y_treino)

## 4) Avaliação do modelo
previsao_y = modelo_knn.predict(X_teste)
previsao_y

### Computando o desempenho

Para fins de curiosidade, vamos comparar o desempenho entre esses dois algoritmos. A função `acuraccy_score` do `scikit-learn` calcula a acurácia das previsões do algoritmo, ou seja, quão eficaz ele é em prever y que são exatamente o y verdadeiro.

In [ ]:
acuracia_knn = accuracy_score(y_teste, y_previsto)
acuracia_knn

In [ ]:
acuracia = accuracy_score(y_test, y_previsto)
acuracia

## Resultados e Discussões

Ao aplicar o algoritmo Naive Bayes em um conjunto de dados real, foi possível observar seu desempenho em comparação com um classificador baseline, revelando aspectos importantes sobre sua capacidade de generalização e sensibilidade às características dos dados. Os resultados mostraram que, embora o modelo baseline ofereça uma referência útil, o algoritmo estudado apresentou maior precisão e melhor separação entre as classes, especialmente em cenários com dados bem distribuídos. Essa análise reforça a importância de entender o funcionamento interno dos algoritmos para fazer escolhas mais informadas em projetos de aprendizado de máquina.

## Conclusões

Este notebook teve como objetivo compreender e aplicar de forma didática um algoritmo de classificação supervisionada, destacando sua fundamentação teórica. Ao induzir um modelo com o algoritmo escolhido e compará-lo com um classificador baseline, foi possível avaliar seu desempenho e entender suas vantagens e limitações em cenários reais. Percebe-se que, ao comparar o desempenho do algoritmo Gaussian Naive Bayes com o desempenho de um algoritmo baseline como o K-NN, obteu-se que a acurácia referente ao nosso Gaussian NB é significantemente maior do que a do KNN. Apesar da lógica simplificada do Gaussian Naive Bayes, ele demonstra-se muito eficiente em realizar previsões bastante próximas a realidade, o que o diferencia de um algoritmo categórico de baseline. Assim, o material contribui para o desenvolvimento de uma habilidade essencial: a capacidade de estudar e aplicar algoritmos não abordados diretamente em sala de aula.


## Referências Bibliográficas
1. IBM. Máquina de vetores de suporte. Disponível em: <https://www.ibm.com/br-pt/think/topics/support-vector-machine>.
2. OPENAI. ChatGPT. São Francisco: OpenAI, 2025. Disponível em: <https://chat.openai.com/>. Acesso em: 10 set. 2025.
3. WHYAMIT404. Creating a Gaussian Distribution with NumPy. Disponível em: <https://medium.com/@whyamit404/creating-a-gaussian-distribution-with-numpy-dc810d931ee0>. Acesso em: 29 set. 2025.
4. RYAN & MATT DATA SCIENCE. Naive Bayes Classifier: A Practical Tutorial with Scikit-Learn. Disponível em: <https://www.youtube.com/watch?v=T6OxPncD7_w>.
5. RYAN & MATT DATA SCIENCE. Normal Distribution in Python: A Beginner’s Guide with Scipy & Numpy. Disponível em: <https://www.youtube.com/watch?v=_Q-CAxnd69U>. Acesso em: 29 set. 2025.
6. Fit Normal Distribution Object. Disponível em: <https://www.mathworks.com/help/stats/normal-distribution.html>.

## Uso de Inteligência Artificial

O modelo Microsoft Copilot foi utilizado especificamente para criar a história inicial no estilo RPG e para revisar os textos de desenvolvimento dos exercícios, ajudando a detalhar o passo a passo seguido para guiar melhor o leitor. As perguntas realizadas à IA foram no estilo: *"Escrevi o seguinte texto: ... Como posso melhorá-lo para guiar o leitor passo a passo, mostrando de maneira clara minha resolução?"*.